<a href="https://colab.research.google.com/github/Shubhranshu-Malhotra/Tensorflow-Developer-ZTM/blob/main/%5BPractice%5D_08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing Basics in TensorFlow


## Get helper functions


In [203]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-05-31 16:29:05--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-05-31 16:29:05 (88.6 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [204]:
# Import helper functions
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Download a text dataset

We'll be using the [Real or Not?](https://www.kaggle.com/c/nlp-getting-started/data) datset from Kaggle which contains text-based Tweets about natural disasters. 


In [205]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2021-05-31 16:29:06--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.26.128, 172.217.193.128, 172.217.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.26.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2021-05-31 16:29:06 (124 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



`nlp_getting_started.zip` contains the following 3 `.csv` files:
* `sample_submission.csv` - an example of the file you'd submit to the Kaggle competition of your model's predictions.
* `train.csv` - training samples of real and not real diaster Tweets.
* `test.csv` - testing samples of real and not real diaster Tweets.

## Visualize text dataset

In [206]:
# Read the data
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [207]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [208]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [209]:
train_df_shuffled = train_df.sample(frac = 1, random_state = 42)

In [210]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [211]:
# Visualize the test data
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [212]:
# How many examples of each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [213]:
# Total samples
len(train_df), len(test_df)

(7613, 3263)

In [214]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [215]:
# Visualize some random training sample
import random
for i in range(10):
  random_index = random.choice(range(0,len(train_df_shuffled)))
  target = train_df.iloc[random_index].target
  text = train_df.iloc[random_index].text
  print(f"Index: {random_index}")
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Index: 3176
Target: 0 (not real disaster)
Text:
Good tips! Does your family have an emergency plan?  ... http://t.co/r5BgVLqPJt http://t.co/MEHWKZwtXD

---

Index: 1397
Target: 1 (real disaster)
Text:
Warfighting Robots Could Reduce Civilian Casualties So Calling for a... http://t.co/9DVU1RidZ3

---

Index: 3481
Target: 1 (real disaster)
Text:
A Time-Lapse Map of Every Nuclear Explosion Since 1945 - by Isao Hashimoto #War #atomicbomb 
https://t.co/V0t8H4Iecc

---

Index: 2869
Target: 1 (real disaster)
Text:
#DroughtMonitor: Moderate or worse #drought ? to ~27% of contig USA; affects ~80M people. http://t.co/YBE9JQoznR http://t.co/328SzflEtZ

---

Index: 7092
Target: 0 (not real disaster)
Text:
@abcnews UK scandal of 2009 caused major upheaval to Parliamentary expenses with subsequent sackings and prison. What are we waiting for?

---

Index: 5296
Target: 1 (real disaster)
Text:
Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legio... http://t.

In [216]:
len(train_df_shuffled)

7613

### Split data into training and validation sets

Since the test set has no labels and we need a way to evalaute our trained models, we'll split off some of the training data and create a validation set.


In [217]:
from sklearn.model_selection import train_test_split

In [218]:
X_train, X_val, y_train, y_val = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                  train_df_shuffled['target'].to_numpy(),
                                                  test_size = 0.1,
                                                  random_state = 42)

In [219]:
len(X_train), len(y_train), len(X_val), len(y_val)

(6851, 6851, 762, 762)

In [220]:
len(train_df_shuffled)

7613

In [221]:
len(X_train) + len(X_val)

7613

In [222]:
# Check the first 10 samples
X_train[:10], y_train[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [223]:
X_val[:10], y_val[:10]

(array(['DFR EP016 Monthly Meltdown - On Dnbheaven 2015.08.06 http://t.co/EjKRf8N8A8 #Drum and Bass #heavy #nasty http://t.co/SPHWE6wFI5',
        'FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/qZQc8WWwcN via @usatoday',
        'Gunmen kill four in El Salvador bus attack: Suspected Salvadoran gang members killed four people and wounded s... http://t.co/CNtwB6ScZj',
        '@camilacabello97 Internally and externally screaming',
        'Radiation emergency #preparedness starts with knowing to: get inside stay inside and stay tuned http://t.co/RFFPqBAz2F via @CDCgov',
        'Investigators rule catastrophic structural failure resulted in 2014 Virg.. Related Articles: http://t.co/Cy1LFeNyV8',
        'How the West was burned: Thousands of wildfires ablaze in #California alone http://t.co/iCSjGZ9tE1 #climate #energy http://t.co/9FxmN0l0Bd',
        "Map: Typhoon Soudelor's predicted path as it approaches Taiwan; expected to make landfall over so

## Converting text into numbers


### Method 1: Text Vecorization (Tokenization) 

In [224]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters [FOR LEARNING PURPOSE]
# Automaticalls adds <OOV> token for unknown words
test_vectorizer = TextVectorization(max_tokens=None, # num of words in vocabulary
                                    standardize = "lower_and_strip_punctuation",
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = 'int',
                                    output_sequence_length = None,
                                    pad_to_max_tokens = True
                                    )


In [225]:
len(X_train[0].split())

7

In [226]:
# Find the average number of tokes (word) in the training tweets
AVERAGE_LEN = round(sum([len(i.split()) for i in X_train])/len(X_train))
AVERAGE_LEN

15

In [227]:
# Set up text vectoriation variables
MAX_VOCAB_LEN = 10000
MAX_LEN = AVERAGE_LEN

In [228]:
text_vectorizer = TextVectorization(max_tokens= MAX_VOCAB_LEN,
                                    output_mode = 'int',
                                    output_sequence_length = MAX_LEN)

In [229]:
# Fit the text vectorizer to the training set
text_vectorizer.adapt(X_train)

In [230]:
type(X_train)

numpy.ndarray

In [231]:
# Create a sample sentence and tokenize it
sample_sentences = ['We had a 6.5 earthquake yesterday.', 'We will go for a picnic tomorrow.']
text_vectorizer(sample_sentences)

<tf.Tensor: shape=(2, 15), dtype=int64, numpy=
array([[  46,   94,    3,    1,  290, 1730,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [  46,   38,  112,   10,    3,    1,  665,    0,    0,    0,    0,
           0,    0,    0,    0]])>

1 represents the \<OOV> token

In [232]:
# Choose a random sentence from training dataset and tokenize it
random_sentence = random.choice(X_train)
print(f'Original text:\n {random_sentence},\
      \n \n Vectorized version:\n {text_vectorizer([random_sentence])}')

Original text:
 I've been meaning to harm you in the best way I see fit??,      
 
 Vectorized version:
 [[ 276   59 5150    5  396   12    4    2  149  147    8   99 3845    0
     0]]


In [233]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in the vocab: {len(words_in_vocab)}")
print(f"5 most common words in vocab: {top_5_words}")
print(f"5 least common words in vocab: {bottom_5_words}")

Number of words in the vocab: 10000
5 most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
5 least common words in vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

The main parameters we're concerned about here are:
* `input_dim` - The size of the vocabulary (e.g. `len(text_vectorizer.get_vocabulary()`).
* `output_dim` - The size of the output embedding vector, for example, a value of `100` outputs a  feature vector of size 100 for each word.
* `embeddings_initializer` - How to initialize the embeddings matrix, default is `"uniform"` which randomly initalizes embedding matrix with uniform distribution. This can be changed for using pre-learned embeddings.
* `input_length` - Length of sequences being passed to embedding layer.


In [234]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = MAX_VOCAB_LEN,
                             output_dim = 128,
                             input_length = MAX_LEN)
embedding

In [235]:
# Get a random sentence from training set and pass through embedding layer
random_sentence = random.choice(X_train)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence])) 
# Right now the representation is random but Will be trained during training to represent sentences better
sample_embed

Original text:
Hellfire! We donÛªt even want to think about it or mention it so letÛªs not do anything that leads to it #islam!      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03394032, -0.00414902, -0.00505615, ...,  0.04956752,
         -0.0431301 , -0.0019838 ],
        [ 0.04411567, -0.03109492, -0.03186083, ...,  0.0035655 ,
          0.03253884, -0.04101529],
        [ 0.01489315,  0.03877788, -0.02588973, ..., -0.00502955,
          0.01819697, -0.01448749],
        ...,
        [ 0.04902091,  0.01188564,  0.00787157, ..., -0.03334521,
          0.01139297,  0.01662776],
        [ 0.03202711,  0.03683296,  0.02506775, ..., -0.01905179,
         -0.02268473,  0.03244457],
        [-0.03981008,  0.04948455,  0.03463351, ..., -0.00662028,
          0.01352016, -0.02012481]]], dtype=float32)>

In [236]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.03394032, -0.00414902, -0.00505615,  0.01295565, -0.02924624,
        -0.01773889, -0.02468914, -0.00222046, -0.04161549,  0.04947766,
         0.0182218 ,  0.02679162, -0.04718339, -0.04511505,  0.04513779,
        -0.01036153, -0.0081498 ,  0.00190473,  0.04820727, -0.00842274,
        -0.00233291, -0.03278758,  0.01559044, -0.00754191, -0.01132131,
         0.023405  , -0.03292253,  0.03924102,  0.03552529,  0.01706677,
         0.04694769,  0.02295091,  0.00053942, -0.02066586, -0.02896285,
         0.03332068,  0.02040577,  0.01101758,  0.00572832,  0.0270206 ,
        -0.00799593, -0.00488629,  0.01216163,  0.01845653,  0.020383  ,
        -0.00632035,  0.03249342,  0.0331453 , -0.02876091,  0.03235031,
         0.0044861 , -0.03440344, -0.01962955,  0.03671552, -0.0076311 ,
        -0.04391711,  0.03377061, -0.02992722,  0.03416386,  0.03173857,
         0.00265987,  0.0492785 ,  0.01538069, -0.00884574, -0.04098377,
  

## Modelling a text dataset (running a series of experiments)
we'll be building the following:
* **Model 0**: Naive Bayes (baseline) 
* **Model 1**: Feed-forward neural network (dense model)
* **Model 2**: LSTM model
* **Model 3**: GRU model
* **Model 4**: Bidirectional-LSTM model
* **Model 5**: 1D Convolutional Neural Network
* **Model 6**: TensorFlow Hub Pretrained Feature Extractor
* **Model 7**: Same as model 6 with 10% of training data


Each experiment will go through the following steps:
* Construct the model
* Train the model
* Make predictions with the model
* Track prediction evaluation metrics for later comparison


### Model 0: Getting a baseline [Naive Bayes]

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.  
  
Model 0 is the simplest to acquire a baseline which we'll expect each other of the other deeper models to beat.  


To create our baseline, we'll create a Scikit-Learn Pipeline using the TF-IDF (term frequency-inverse document frequency) formula to convert our words to numbers and then model them with the [Multinomial Naive Bayes algorithm](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB). This was chosen via referring to the [Scikit-Learn machine learning map](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html).


In [237]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [238]:
# Evaluate our baseline model
baseline_score = model_0.score(X_val, y_val)
print(f"Our baseline model achieves the accuracy of: {baseline_score*100:0.2f}%")

Our baseline model achieves the accuracy of: 79.27%


In [239]:
# Make predictions 
baseline_preds = model_0.predict(X_val)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

* Accuracy
* Precision
* Recall
* F1-score


In [240]:
# add to helper functions
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, accuracy_score
def get_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """

  metrics = {}
  metrics['accuracy'] = 100 * accuracy_score(y_true, y_pred)
  # Calculate model precision, recall and f1 score using "weighted" average
  metrics['precision'], metrics['recall'], metrics['f1_score'], _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  return metrics

In [241]:
baseline_results = get_results(y_val, baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1_score': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 1: A simple dense model


In [242]:
# tensorboard callback
from helper_functions import create_tensorboard_callback
tensorboard_dir = "/content/drive/MyDrive/ZTM_tensorflow_developer/tensorboard"

In [243]:
# Build a model using functional api
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string, name = 'input_layer')
x = text_vectorizer(inputs)
x = embedding(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_1 = tf.keras.Model(inputs, outputs)

In [244]:
model_1.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 1)]               0         
_________________________________________________________________
text_vectorization_4 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
dense_9 (Dense)              (None, 15, 1)             129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [245]:
# Compile the model
model_1.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [246]:
# Fit the model
model_1_history = model_1.fit(X_train, y_train,
                            epochs = 5,
                            validation_data = (X_val, y_val),
                            callbacks = [create_tensorboard_callback(dir_name = tensorboard_dir,
                                                                     experiment_name = 'nlp_model_1_dense')])

Saving TensorBoard log files to: /content/drive/MyDrive/ZTM_tensorflow_developer/tensorboard/nlp_model_1_dense/20210531-162909
Epoch 1/5
215/215 [==============================] - 5s 17ms/step - loss: 0.6493 - accuracy: 0.6390 - val_loss: 0.6293 - val_accuracy: 0.6481
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.5971 - accuracy: 0.6894 - val_loss: 0.6244 - val_accuracy: 0.6480
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.5771 - accuracy: 0.6905 - val_loss: 0.6277 - val_accuracy: 0.6482
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.5678 - accuracy: 0.6932 - val_loss: 0.6311 - val_accuracy: 0.6459
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.5637 - accuracy: 0.6918 - val_loss: 0.6313 - val_accuracy: 0.6463


In [247]:
# Check the results
model_1.evaluate(X_val, y_val)

24/24 [==============================] - 0s 2ms/step - loss: 0.6313 - accuracy: 0.6463


[0.6312629580497742, 0.6462817192077637]

In [248]:
# Make predictions
model_1_pred_probs = model_1.predict(X_val)
model_1_pred_probs, model_1_pred_probs.shape

(array([[[0.45258433],
         [0.45258433],
         [0.45258433],
         ...,
         [0.03935131],
         [0.45258433],
         [0.39245868]],
 
        [[0.5742079 ],
         [0.38198924],
         [0.46945179],
         ...,
         [0.8217069 ],
         [0.45258433],
         [0.566287  ]],
 
        [[0.931408  ],
         [0.53375846],
         [0.5857269 ],
         ...,
         [0.9689521 ],
         [0.5857269 ],
         [0.54292756]],
 
        ...,
 
        [[0.5742079 ],
         [0.3562048 ],
         [0.38198924],
         ...,
         [0.8217069 ],
         [0.45258433],
         [0.39245868]],
 
        [[0.0982407 ],
         [0.6179534 ],
         [0.41631907],
         ...,
         [0.41414636],
         [0.2991696 ],
         [0.3486641 ]],
 
        [[0.1770744 ],
         [0.14827904],
         [0.45884922],
         ...,
         [0.43125758],
         [0.2883792 ],
         [0.6129771 ]]], dtype=float32), (762, 15, 1))

In [249]:
model_1_pred_probs[0]

array([[0.45258433],
       [0.45258433],
       [0.45258433],
       [0.18486896],
       [0.49349102],
       [0.45258433],
       [0.45258433],
       [0.45258433],
       [0.10623497],
       [0.4148156 ],
       [0.45258433],
       [0.95250505],
       [0.03935131],
       [0.45258433],
       [0.39245868]], dtype=float32)

There is something wrong here as we only need one probability for the whole sentence and not probability for each token.  
This problem is caused because we didn't use GlobalAveragePooling.  
Let's rebuild it, this time adding the GlobalAveragePooling layer.  

In [269]:
# Build a model using functional api
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string, name = 'input_layer')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_1 = tf.keras.Model(inputs, outputs)


In [251]:
model_1.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 1)]               0         
_________________________________________________________________
text_vectorization_4 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d_6 ( (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [252]:
# Compile the model
model_1.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [253]:
# Fit the model
model_1_history = model_1.fit(X_train, y_train,
                            epochs = 5,
                            validation_data = (X_val, y_val),
                            callbacks = [create_tensorboard_callback(dir_name = tensorboard_dir,
                                                                     experiment_name = 'nlp_model_1_dense')])

Saving TensorBoard log files to: /content/drive/MyDrive/ZTM_tensorflow_developer/tensorboard/nlp_model_1_dense/20210531-162927
Epoch 1/5
215/215 [==============================] - 4s 17ms/step - loss: 0.5663 - accuracy: 0.7609 - val_loss: 0.5204 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3982 - accuracy: 0.8529 - val_loss: 0.4618 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3169 - accuracy: 0.8787 - val_loss: 0.4500 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2642 - accuracy: 0.9024 - val_loss: 0.4604 - val_accuracy: 0.7979
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2243 - accuracy: 0.9184 - val_loss: 0.4664 - val_accuracy: 0.7822


In [254]:
# Check the results
model_1.evaluate(X_val, y_val)

24/24 [==============================] - 0s 2ms/step - loss: 0.4664 - accuracy: 0.7822


[0.4663676917552948, 0.7821522355079651]

In [255]:
# Make predictions
model_1_pred_probs = model_1.predict(X_val)
model_1_pred_probs.shape

(762, 1)

In [256]:
model_1_pred_probs[0]

array([0.41683644], dtype=float32)

In [257]:
# look at first 10 predictions
model_1_pred_probs[:10]

array([[0.41683644],
       [0.7686113 ],
       [0.9970106 ],
       [0.18166536],
       [0.14640331],
       [0.9493388 ],
       [0.88575554],
       [0.9962744 ],
       [0.96875775],
       [0.32348743]], dtype=float32)

In [258]:
y_val[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

As we can see that our validation labels are in 1 or 0 format we need to convert our model's prediction probabilities to label format too.

In [259]:
# Convert prediction probabilities to label format
# model_1_pred_labels = [0 if i>0.5 else 1 for i in model_1_pred_probs]
model_1_pred_labels = tf.squeeze(tf.round(model_1_pred_probs))

In [260]:
model_1_pred_labels[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [261]:
model_1_results = get_results(y_val, model_1_pred_labels)
model_1_results

{'accuracy': 78.21522309711287,
 'f1_score': 0.7804155135464126,
 'precision': 0.7835553063606987,
 'recall': 0.7821522309711286}

In [262]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1_score': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [263]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualizing learned embeddings

Our first model (`model_1`) contained an embedding layer (`embedding`) which learned a way of representing words as feature vectors by passing over the training data.

To further help understand what a text embedding is, let's visualize the embedding our model learned.

To do so, let's remind ourselves of the words in our vocabulary.


In [264]:
# Get the vocabulary
word_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [265]:
MAX_VOCAB_LEN

10000

In [270]:
model_1.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 1)]               0         
_________________________________________________________________
text_vectorization_4 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d_7 ( (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [272]:
# Get the weight matrix of embedding layer 
# (these are the numerical patterns between the text in the training dataset the model has learned)
embed_weights = model_1.get_layer('embedding_2').get_weights()[0]

In [273]:
embed_weights

array([[ 0.000851  , -0.06826828,  0.0104061 , ..., -0.03496656,
        -0.05664244,  0.01203023],
       [-0.01925762,  0.0171201 ,  0.01295353, ..., -0.02738079,
         0.00070074, -0.07143973],
       [-0.01727312, -0.06529462,  0.01217169, ...,  0.01467015,
        -0.04821666, -0.04728634],
       ...,
       [ 0.0090167 ,  0.01473334,  0.01535289, ...,  0.04191733,
        -0.04135593,  0.046917  ],
       [ 0.03651113,  0.10984478,  0.03795978, ..., -0.07329595,
         0.07839   ,  0.12091658],
       [ 0.14036895,  0.10980735,  0.12360164, ..., -0.08522652,
         0.08948015,  0.09086819]], dtype=float32)

In [274]:
embed_weights.shape

(10000, 128)

Now we've got these two objects, we can use the [Embedding Projector tool](http://projector.tensorflow.org/_) to visualize our embedding. 

To use the Embedding Projector tool, we need two files:
* The embedding vectors (same as embedding weights).
* The meta data of the embedding vectors (the words they represent - our vocabulary).

Right now, we've got of these files as Python objects. To download them to file, we're going to [use the code example available on the TensorFlow word embeddings tutorial page](https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk).


In [275]:
# Code below is adapted from: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
import io

# Create output writers
out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# Write embedding vectors and words to file
for num, word in enumerate(words_in_vocab):
  if num == 0: 
     continue # skip padding token
  vec = embed_weights[num]
  out_m.write(word + "\n") # write words to file
  out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
out_v.close()
out_m.close()

# Download files locally to upload to Embedding Projector
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download("embedding_vectors.tsv")
  files.download("embedding_metadata.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>